# 使用来源进行检索性问答

本文档介绍如何利用索引进行带来源的问答。它是通过使用`RetrievalQAWithSourcesChain`来实现的，该工具可以从索引中查找文档。

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma

In [2]:
with open("../../state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [3]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": f"{i}-pl"} for i in range(len(texts))])

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [4]:
from langchain.chains import RetrievalQAWithSourcesChain

In [5]:
from langchain import OpenAI

chain = RetrievalQAWithSourcesChain.from_chain_type(OpenAI(temperature=0), chain_type="stuff", retriever=docsearch.as_retriever())

In [6]:
chain({"question": "What did the president say about Justice Breyer"}, return_only_outputs=True)

{'answer': ' The president honored Justice Breyer for his service and mentioned his legacy of excellence.\n',
 'sources': '31-pl'}

## 链式类型
您可以轻松指定不同的链式类型来加载和使用RetrievalQAWithSourcesChain链。关于这些类型的更详细说明，请参见[此笔记本](qa_with_sources.ipynb)。

有两种方法可以加载不同的链式类型。首先，您可以在`from_chain_type`方法中指定链式类型参数。这允许您传递要使用的链式类型的名称。例如，在下面的示例中，我们将链式类型更改为`map_reduce`。

In [7]:
chain = RetrievalQAWithSourcesChain.from_chain_type(OpenAI(temperature=0), chain_type="map_reduce", retriever=docsearch.as_retriever())

In [8]:
chain({"question": "What did the president say about Justice Breyer"}, return_only_outputs=True)

{'answer': ' The president said "Justice Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service."\n',
 'sources': '31-pl'}

上述方法使您可以非常简单地更改chain_type，但它确实在链式类型的参数上提供了大量的灵活性。如果您想控制这些参数，可以直接加载链式类型（就像您在[此笔记本](qa_with_sources.ipynb)中所做的那样），然后将其直接传递给RetrievalQAWithSourcesChain链的`combine_documents_chain`参数。例如：

In [10]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
qa_chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
qa = RetrievalQAWithSourcesChain(combine_documents_chain=qa_chain, retriever=docsearch.as_retriever())

In [11]:
qa({"question": "What did the president say about Justice Breyer"}, return_only_outputs=True)

{'answer': ' The president honored Justice Breyer for his service and mentioned his legacy of excellence.\n',
 'sources': '31-pl'}